In [1]:
import random
from bolib.Bo import Bo
from bolib.Dimension import NumericDimension
from bolib.ComputeSpace import ComputeSpace
from bolib.Normalizer import OptimizeFor
import pandas as pd
from IPython.display import clear_output
import numpy as np

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blackbox_dims = 10

In [3]:

def getRating(v, target):    
    return [sum([1 for i in v if i in target])]
    
def getSample(TARGET = None):
    s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]

    if TARGET is not None:
        
        r= getRating(s, TARGET)
        return [s, r]
    return s


TARGET = getSample()

s, r = getSample(TARGET)
pd.DataFrame([[s, r, TARGET]], columns=["guess", "rating", "target"])  

,guess,rating,target
0,"[9, 18, 20, 34, 42, 53, 60, 72, 82, 98]",[1],"[6, 11, 23, 32, 49, 51, 67, 77, 83, 98]"


In [4]:
[ [i*10+j for j in range(10)] for i in range(blackbox_dims) ]
s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]
s

[3, 14, 23, 33, 40, 50, 68, 72, 88, 93]

In [5]:
from bolib.Dimension import DiscreteDimension


numerics = [ DiscreteDimension(elements=[i*10+j for j in range(10)]  ,name=f"p{i}") for i in range(blackbox_dims) ]
ranking = [ NumericDimension(min=0, max=10, name="Ranking", optimize_for=OptimizeFor.MIN) ]



def infer(x, y, bo, cs):  
  cs.add_values(xs = [x], ys=[y])   

  inf = bo.infer()
  denorm = cs.denormalize(inf)
  return [d.value for d in denorm]





In [6]:
for i in numerics:
  print(i._elements)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [7]:
compSpace = ComputeSpace(numerics, ranking)
bo = Bo(compSpace)

last_rating = 0
last_guess = None
best_guess = None
best_rating = None
print(f"We are looking for {TARGET}")

df_cols = ["guess", "rating", "target"]
df = pd.DataFrame([], columns=df_cols)  

for i in range(50):
  last_rating = r[0]
  last_guess = s
  
  
  guess = infer(s,r, bo, compSpace)  
  print("guess", guess)
  r = getRating(guess, TARGET)

  if r[0] == 0:
    guess = getSample()

  tdf = pd.DataFrame([[s, r, TARGET]], columns=df_cols)  
  
  df = pd.concat([df, tdf], axis=0).reset_index(drop=True)
  
  clear_output(wait=True)

  print(df)

  
  #if r[0] == 2 
  """
  if r[0] < 0.15:
    print(f"Found it in {i} steps: {guess}")
    break
  """
  if best_guess is None:
    best_rating = r[0]
    best_guess = guess

  if r[0] > best_rating:
    best_guess = guess
    best_rating = r[0]
  s = guess

print(f"I think it is {best_guess} with a rating of {best_rating}")


  


  



                                      guess rating  \
0   [3, 14, 23, 33, 40, 50, 68, 72, 88, 93]    [1]   
1   [3, 19, 29, 31, 44, 58, 60, 79, 83, 99]    [1]   
2   [7, 18, 20, 33, 49, 59, 69, 79, 88, 92]    [1]   
3   [9, 12, 28, 39, 49, 59, 61, 71, 88, 92]    [3]   
4   [7, 13, 28, 32, 40, 59, 67, 70, 81, 98]    [2]   
5   [6, 17, 21, 33, 47, 56, 68, 77, 88, 92]    [1]   
6   [9, 19, 20, 33, 49, 59, 69, 79, 88, 92]    [1]   
7   [7, 19, 20, 33, 49, 59, 69, 79, 88, 92]    [0]   
8   [1, 13, 23, 36, 49, 50, 63, 74, 85, 94]    [1]   
9   [0, 12, 24, 37, 49, 50, 62, 73, 85, 94]    [1]   
10  [2, 14, 22, 35, 49, 50, 64, 75, 85, 94]    [1]   
11  [1, 13, 23, 36, 48, 50, 63, 73, 85, 94]    [1]   
12  [0, 12, 24, 37, 49, 50, 62, 74, 85, 94]    [2]   
13  [1, 13, 23, 36, 49, 50, 63, 74, 85, 94]    [0]   
14  [8, 16, 21, 32, 41, 52, 68, 79, 84, 92]    [3]   
15  [8, 16, 21, 32, 40, 51, 68, 79, 83, 92]    [1]   
16  [9, 16, 21, 32, 42, 54, 68, 79, 86, 92]    [0]   
17  [6, 14, 24, 30, 45, 58, 

In [8]:
print(f"I think it is \n{best_guess} with a rating of {best_rating}")
print(TARGET)


I think it is 
[1, 12, 21, 32, 42, 50, 67, 71, 83, 98] with a rating of 4
[6, 11, 23, 32, 49, 51, 67, 77, 83, 98]


  name  value                                     space  normalized
0   p0      6            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]         0.6
1   p1     17  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]         0.7
2   p2     21  [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]         0.1
3   p3     33  [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]         0.3
4   p4     47  [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]         0.7
5   p5     56  [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]         0.6
6   p6     68  [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]         0.8
7   p7     77  [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]         0.7
8   p8     88  [80, 81, 82, 83, 84, 85, 86, 87, 88, 89]         0.8
9   p9     92  [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]         0.2